# Data Crawling

In [2]:
import requests
import pandas as pd

def get_binance_ohlcv(symbol="BTCUSDT", interval="1h", start="2021-01-01", end="2026-02-19"):
    url = "https://fapi.binance.com/fapi/v1/klines"
    start_ms = int(pd.Timestamp(start).timestamp() * 1000)
    end_ms   = int(pd.Timestamp(end).timestamp() * 1000)
    
    all_data = []
    while start_ms < end_ms:
        params = {"symbol": symbol, "interval": interval,
                  "startTime": start_ms, "endTime": end_ms, "limit": 1000}
        resp = requests.get(url, params=params).json()
        if not resp:
            break
        all_data.extend(resp)
        start_ms = resp[-1][0] + 1  # move to next timestamp for next candle

    df = pd.DataFrame(all_data, columns=[
        "open_time","open","high","low","close","volume",
        "close_time","quote_volume","trades",
        "taker_buy_base","taker_buy_quote","ignore"
    ])
    df["open_time"] = pd.to_datetime(df["open_time"], unit="ms")
    df[["open","high","low","close","volume"]] = df[["open","high","low","close","volume"]].astype(float)
    return df

df = get_binance_ohlcv()
# df.to_csv("binance_btcusdt_1h.csv", index=False)

# Data Cleaning

In [3]:
# df = df.isna().sum()
ohlc_violations = (
    (df["high"] < df["low"]) |       
    (df["high"] < df["open"]) |         
    (df["high"] < df["close"]) |          
    (df["low"] > df["open"]) |            
    (df["low"] > df["close"])             
)
ohlc_violations_count = ohlc_violations.sum()
print(f"Number of OHLC violations: {ohlc_violations_count}")

df["open_time"] = pd.to_datetime(df["open_time"], unit="hrs")
df = df.rename(columns={"open_time": "timestamp"})
df.drop(columns=["close_time", "quote_volume", "trades", "taker_buy_base", "taker_buy_quote", "ignore"], inplace=True)

Number of OHLC violations: 0


# Alpha Definition

### Alpha A (Geometric mean of the bar minus VWAP)

In [4]:
import numpy as np

# how far it is the geometric mean is from VWAP

typical_price = (df["high"] + df["low"] + df["close"]) / 3  # encode the information of the bar biased towards closing price
df["VWAP"] = (typical_price * df["volume"]).rolling(24).sum() / df["volume"].rolling(24).sum()
df["alpha_A"] = np.sqrt(df["high"] * df["low"]) - df["VWAP"] # geometric mean of the bar minus VWAP

### Alpha B (where the bar close divided by amplified open-to-close ratio)

In [5]:
# alpha_B = -(C - L)/(H - L) × (O/C)^5        [0,-inf)
# bar close high -> (C - L)/(H - L) close to 1, bar close low -> (C - L)/(H - L) close to 0, bar close middle -> (C - L)/(H - L) close to 0.5
# final value more negative -> more bullish, final value closer to 0 -> more bearish

df["alpha_B"] = -1 * ((df["low"] - df["close"]) * (df["open"] ** 5))/((df["low"]-df["high"])*(df["close"]**5))

### Alpha C (candle body to range ratio)

In [6]:
# close to 1 -> very bullish
# close to 0 -> doji
# close to -1 -> very bearish

df["alpha_C"] = (df["close"] - df["open"]) / (df["high"] - df["low"]) + 0.001

In [7]:
df.tail()

,timestamp,open,high,low,close,volume,VWAP,alpha_A,alpha_B,alpha_C
44996,2026-02-18 20:00:00,66138.3,66528.8,66098.9,66230.2,7627.587,67246.441195,-932.939566,-0.303307,0.214771
44997,2026-02-18 21:00:00,66230.3,66466.8,65993.1,66300.6,5568.422,67206.724016,-977.197526,-0.645711,0.149406
44998,2026-02-18 22:00:00,66300.5,66451.5,66192.7,66379.0,1860.076,67193.476189,-871.502424,-0.715614,0.304323
44999,2026-02-18 23:00:00,66379.0,66472.6,66240.0,66422.8,2506.522,67177.334875,-821.136793,-0.783311,0.189306
45000,2026-02-19 00:00:00,66422.8,66573.6,66301.8,66410.7,2414.967,67163.378280,-725.817274,-0.401027,-0.043518


# Backtesting

### Data Split

In [42]:
# Drop the data that are too old
df_clean = df[df["timestamp"] >= "2024-01-01"].reset_index(drop=True)

split_index = int(len(df_clean) * 0.5)
backtest_df = df_clean.iloc[:split_index].copy()
test_df = df_clean.iloc[split_index:].copy()

In [43]:
def run_backtest(df, lookback=24,alpha_1_threshold=1, alpha_2_threshold=1):
    alpha_A_z = (df["alpha_A"] - df["alpha_A"].rolling(lookback).mean())/df["alpha_A"].rolling(lookback).std()
    alpha_C_z = (df["alpha_C"] - df["alpha_C"].rolling(lookback).mean())/df["alpha_C"].rolling(lookback).std()
    
    signal = pd.Series(0, index=df.index)
    signal[(alpha_A_z >= alpha_1_threshold) & (alpha_C_z <= -alpha_2_threshold)] = -1  # Short
    signal[(alpha_A_z <= -alpha_1_threshold) & (alpha_C_z >= alpha_2_threshold)] = 1   # Long

    returns = df["close"].pct_change().shift(-1) * signal
    return returns


lookback_range = [2, 4, 6, 12, 24, 48, 72, 96, 120, 144]
alpha_1_threshold_range = np.arange(0.3, 4, 0.1)  
alpha_2_threshold_range = np.arange(0.3, 4, 0.1)  

results = []
for lookback in lookback_range:
    for alpha_1_threshold in alpha_1_threshold_range:
        for alpha_2_threshold in alpha_2_threshold_range:
            backtest_returns = run_backtest(backtest_df, lookback, alpha_1_threshold, alpha_2_threshold)
            sharpe_ratio = backtest_returns.mean() / backtest_returns.std() * np.sqrt(8760)
            results.append((lookback, alpha_1_threshold, alpha_2_threshold, sharpe_ratio))

results_df = pd.DataFrame(results, columns=["lookback", "alpha_1_threshold", "alpha_2_threshold", "sharpe_ratio"])
best_params = results_df.loc[results_df["sharpe_ratio"].idxmax()]
print("Best Parameters:")
print(best_params)


C:\Users\User\AppData\Local\Temp\ipykernel_7452\1291240500.py:22: RuntimeWarning: invalid value encountered in scalar divide
  sharpe_ratio = backtest_returns.mean() / backtest_returns.std() * np.sqrt(8760)


Best Parameters:
lookback             4.000000
alpha_1_threshold    1.200000
alpha_2_threshold    1.100000
sharpe_ratio         2.052196
Name: 1710, dtype: float64


## Forward Test

In [44]:
test_result = run_backtest(test_df, int(best_params["lookback"]), best_params["alpha_1_threshold"], best_params["alpha_2_threshold"])

# Annualized Sharpe Ratio
sharpe_ratio = test_result.mean() / test_result.std() * np.sqrt(8760)
print(f"sharpe_ratio: {sharpe_ratio:.2f}")

# maximum drawdown
cumulative_returns = (1 + test_result).cumprod()
rolling_max = cumulative_returns.cummax()
drawdown = cumulative_returns / rolling_max - 1
max_drawdown = drawdown.min()
print(f"Maximum Drawdown: {max_drawdown:.2%}")

sharpe_ratio: 0.65
Maximum Drawdown: -4.02%


# Performance Visualization

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates


cumulative_returns = (1 + test_result).cumprod()